Random Forest is an ensemble type of model which consists of many other models like Decision Trees and etc.

Random Sampling of Observations(Bagging) and Random Subset of Features.(if you don't know these, read it up in Random Forest Regression notebook)

In [0]:
%%HTML
<iframe width="560" height="315" src="https://www.youtube.com/embed/loNcrMjYh64" frameborder="0" allow="accelerometer; autoplay; encrypted-media; gyroscope; picture-in-picture" allowfullscreen></iframe>

Random Forest Classifier in Python

In [0]:
import pandas as pd

In [0]:
transact_dataset = pd.read_csv('creditcard.csv')

The datasets contains transactions made by credit cards in September 2013 by european cardholders. This dataset presents transactions that occurred in two days, where we have 492 frauds out of 284,807 transactions.<br>
Unfortunately, due to confidentiality issues, we cannot provide the original features and more background information about the data. Features V1, V2, ... V28 are the principal components obtained with PCA, the only features which have not been transformed with PCA are 'Time' and 'Amount'. Feature 'Time' contains the seconds elapsed between each transaction and the first transaction in the dataset. 

In [0]:
transact_dataset.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


In [0]:
transact_dataset.isnull().any()

Time      False
V1        False
V2        False
V3        False
V4        False
V5        False
V6        False
V7        False
V8        False
V9        False
V10       False
V11       False
V12       False
V13       False
V14       False
V15       False
V16       False
V17       False
V18       False
V19       False
V20       False
V21       False
V22       False
V23       False
V24       False
V25       False
V26       False
V27       False
V28       False
Amount    False
Class     False
dtype: bool

As we can see, none of the columns have empty values. Apparently, the data was normlizied in order to use for machine learning models

Splitting dataset into features and labels. features x=v1 to v28 and amount and label y= class.

In [0]:
X, y = transact_dataset.drop("Class", axis=1), transact_dataset["Class"]
print(X)


            Time         V1         V2        V3        V4        V5  \
0            0.0  -1.359807  -0.072781  2.536347  1.378155 -0.338321   
1            0.0   1.191857   0.266151  0.166480  0.448154  0.060018   
2            1.0  -1.358354  -1.340163  1.773209  0.379780 -0.503198   
3            1.0  -0.966272  -0.185226  1.792993 -0.863291 -0.010309   
4            2.0  -1.158233   0.877737  1.548718  0.403034 -0.407193   
5            2.0  -0.425966   0.960523  1.141109 -0.168252  0.420987   
6            4.0   1.229658   0.141004  0.045371  1.202613  0.191881   
7            7.0  -0.644269   1.417964  1.074380 -0.492199  0.948934   
8            7.0  -0.894286   0.286157 -0.113192 -0.271526  2.669599   
9            9.0  -0.338262   1.119593  1.044367 -0.222187  0.499361   
10          10.0   1.449044  -1.176339  0.913860 -1.375667 -1.971383   
11          10.0   0.384978   0.616109 -0.874300 -0.094019  2.924584   
12          10.0   1.249999  -1.221637  0.383930 -1.234899 -1.48

In [0]:
#it shows 492 transactions are wrong
print(y)
print(sum(y))


0         0
1         0
2         0
3         0
4         0
5         0
6         0
7         0
8         0
9         0
10        0
11        0
12        0
13        0
14        0
15        0
16        0
17        0
18        0
19        0
20        0
21        0
22        0
23        0
24        0
25        0
26        0
27        0
28        0
29        0
         ..
284777    0
284778    0
284779    0
284780    0
284781    0
284782    0
284783    0
284784    0
284785    0
284786    0
284787    0
284788    0
284789    0
284790    0
284791    0
284792    0
284793    0
284794    0
284795    0
284796    0
284797    0
284798    0
284799    0
284800    0
284801    0
284802    0
284803    0
284804    0
284805    0
284806    0
Name: Class, Length: 284807, dtype: int64
492


Splitting features and labels into Train and Test split 

In [0]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=0)

Creating the Random Forest Classifier model and fitting the Training set

In [0]:
from sklearn.ensemble import RandomForestClassifier
rf_clas_1 = RandomForestClassifier(random_state=0)
rf_clas_1.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=0, verbose=0, warm_start=False)

Predicting Test set 

In [0]:
y_pred = rf_clas_1.predict(X_test)

In [0]:
from sklearn.metrics import confusion_matrix

Creating confusion matrix to evaluate the model's performance

In [0]:
cm = confusion_matrix(y_test, y_pred)
cm

array([[93817,     8],
       [   41,   121]], dtype=int64)

Interpreting the Confusion Matrix:
<br>Model predicted 93817 as non-fraudulent correctly.
<br>Model predicted 121 as fraudulent correctly.
<br>Model made a mistake predicting 8 transactions as fraudulent, but they were not.
<br>Model made a mistake predicting 41 transactions as non-fraudulent, but they were.

In [0]:
len(y_test[y_test==1])

162

The dataset was quite imbalanced because of the proportion of fraudulent and non-fraudulent transactions. There were only 492 out of 284,807 transactions.

That's why I will evaluate the model based on only Fraudulent transactions

There were only 145 fraudulent transactions in testing set and 112 of them were predicted correctly 

In [0]:
eval_1 = 121/162
eval_1


0.7469135802469136

Now, let's tune the parameters of the model in order to get better prediction 

Getting important features

In [0]:
feature_imp = pd.DataFrame(rf_clas_1.feature_importances_, index=X_test.columns, columns=["Importance"])
print(feature_imp.sort_values(['Importance']))


        Importance
V23       0.006436
V27       0.006828
V13       0.007511
V25       0.008177
V22       0.009194
Amount    0.010028
V24       0.010364
V5        0.010528
V28       0.010655
V8        0.010898
V18       0.011135
V20       0.011276
V19       0.013398
V1        0.014546
V15       0.015744
V21       0.016539
V4        0.017394
V3        0.019027
V26       0.020087
Time      0.020209
V6        0.021849
V2        0.022113
V7        0.026987
V9        0.033672
V16       0.043477
V10       0.079687
V11       0.093105
V12       0.099886
V14       0.137389
V17       0.191859


Creating another Random Forest Classifier model

In [0]:
rf_clas_2 = RandomForestClassifier(n_estimators=100, criterion="entropy", random_state=0)

In [0]:
rf_clas_2.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='entropy',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=0, verbose=0,
                       warm_start=False)

Predicting test set

In [0]:
y_pred_2 = rf_clas_2.predict(X_test)

Evaluating model's performance

In [0]:
cm_2 = confusion_matrix(y_test, y_pred_2)
cm_2

array([[93818,     7],
       [   37,   125]], dtype=int64)

In [0]:
rf_clas_3 = RandomForestClassifier(n_estimators=100, random_state=0)

In [0]:
rf_clas_3.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=0, verbose=0,
                       warm_start=False)

In [0]:
y_pred_3 = rf_clas_2.predict(X_test)

In [0]:
cm_3 = confusion_matrix(y_test, y_pred_3)
cm_3

array([[93818,     7],
       [   37,   125]], dtype=int64)

There were 100 trees for second and third models, but with different criterion. However, both "gini" and "entropy" criterions has the same result. 

Let's get the proportion of correctly predicted values for frauds only.

In [0]:
eval_3 = 125/162
eval_3

0.7716049382716049

In [0]:
eval_1

0.7469135802469136

We were able to improve the prediction score of the Fraudulent transactions by 3% after tuning the parameters